In [2]:
run TweetTellsAStory.py

hej


In [3]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
import pandas as pd

In [5]:
Trump = pd.read_csv("Trump.csv")

In [6]:
Trump.head(10)

,Date,Time,Tweet_Text,Type,Media_Type,Hashtags,Tweet_Id,Tweet_Url,twt_favourites_IS_THIS_LIKE_QUESTION_MARK,Retweets,Unnamed: 10,Unnamed: 11
0,16-11-11,15:26:37,Today we express our deepest gratitude to all ...,text,photo,ThankAVet,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,127213,41112,NaN,NaN
1,16-11-11,13:33:35,Busy day planned in New York. Will soon be mak...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,141527,28654,NaN,NaN
2,16-11-11,11:14:20,Love the fact that the small groups of protest...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,183729,50039,NaN,NaN
3,16-11-11,2:19:44,Just had a very open and successful presidenti...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,214001,67010,NaN,NaN
4,16-11-11,2:10:46,A fantastic day in D.C. Met with President Oba...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,178499,36688,NaN,NaN
5,16-11-10,19:31:27,Happy 241st birthday to the U.S. Marine Corps!...,text,photo,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,159176,44655,NaN,NaN
6,16-11-09,11:36:58,Such a beautiful and important evening! The fo...,text,NaN,NaN,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,627615,225164,NaN,NaN
7,16-11-09,2:48:27,Watching the returns at 9:45pm.\n#ElectionNigh...,text,photo,ElectionNight;MAGA,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,185160,45492,NaN,NaN
8,16-11-09,1:35:15,RT @IvankaTrump: Such a surreal moment to vote...,text,NaN,NaN,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,99809,17169,NaN,NaN
9,16-11-08,23:20:39,RT @EricTrump: Join my family in this incredib...,text,NaN,MakeAmericaGreatAgain;VOTE,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,63868,19710,NaN,NaN


In [7]:
Trump.Tweet_Text

0       Today we express our deepest gratitude to all ...
1       Busy day planned in New York. Will soon be mak...
2       Love the fact that the small groups of protest...
3       Just had a very open and successful presidenti...
4       A fantastic day in D.C. Met with President Oba...
5       Happy 241st birthday to the U.S. Marine Corps!...
6       Such a beautiful and important evening! The fo...
7       Watching the returns at 9:45pm.\n#ElectionNigh...
8       RT @IvankaTrump: Such a surreal moment to vote...
9       RT @EricTrump: Join my family in this incredib...
10      RT @DonaldJTrumpJr: FINAL PUSH! Eric and I doi...
11      Still time to #VoteTrump!\n#iVoted #ElectionNi...
12      Dont let up, keep getting out to vote - this e...
13      Just out according to @CNN: "Utah officials re...
14      I will be watching the election results from T...
15      #ElectionDay https://t.co/MXrAxYnTjY https://t...
16      We need your vote. Go to the POLLS! Lets conti...
17      VOTE T

In [68]:
trimSet = Trump.Tweet_Text
trimSetLower = trimSet.str.lower()

In [69]:
trimList = ['https']
for i,line in enumerate(trimSetLower):
    for trim in trimList:
        if trim in line:
            print(trim)
            trimSetLower.drop(i,inplace=True)

In [75]:
trimSet.describe()

count                          5094
unique                         5083
top       MAKE AMERICA GREAT AGAIN!
freq                              9
Name: Tweet_Text, dtype: object

In [98]:
raw_text = trimSetLower.str.cat(sep=' ')
raw_text = raw_text[1:round(len(raw_text)/2)]

In [99]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [100]:
n_chars = len(raw_text)
n_vocab = len(chars)
print('Total Characters: ', n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  302565
Total Vocab:  66


In [101]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  302535


In [102]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [103]:
# define the LSTM model
model = Sequential()
model.add(LSTM(16, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [104]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [105]:
model.fit(X, y, epochs=3, batch_size=128, callbacks=callbacks_list)

Epoch 1/3
302535/302535 [==============================] - 169s - loss: 3.1323   
Epoch 2/3
302535/302535 [==============================] - 167s - loss: 3.0276   
Epoch 3/3
302535/302535 [==============================] - 166s - loss: 2.9711   


In [106]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [110]:
# pick a random seed
import sys
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(280):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" ned to that 30% ceiling "@ilea "
  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe  aoe
Done.
